## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [122]:
import pandas
import numpy
numpy.random.seed(2012265)

In [20]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2020-11-15 17:04:38--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.1, 2620:100:6025:1::a27d:4501
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.1|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 301 Moved Permanently
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2020-11-15 17:04:38--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://ucc1b7ac7e87ce6d71a11736731d.dl.dropboxusercontent.com/cd/0/inline/BDRe65D4vUoDYCm7nvSGwDjp6OHiYuzm1_-XY2clkPsEz7HpWftPdNKIXF8d6YLIbP68gurE5YmD2JkCis7HLPu9Nhtul_khx5T6CckmEqU-1bLrQ7Dsz-AXfFzbR9hO7Bk/file# [segue]
--2020-11-15 17:04:39--  https://ucc1b7ac7e87ce6d71a11736731d.dl.dropboxusercontent.com/cd/0/inline/BDRe65D4vUoDYCm7nvSGwDjp6OHiYuzm1_-XY2clkPsEz7HpWftPdNKIXF8d6YLIbP68gurE5YmD

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [39]:
source_file = "./data/data_000637.txt"
lines = open(source_file)
N = min(len(list(lines)), 100000)
lines.close()

data = pandas.read_csv(source_file, header=0, nrows=N)

data.set_index(['FPGA', 'TDC_CHANNEL'], inplace=True)
data.sort_values(['TDC_MEAS','BX_COUNTER','ORBIT_CNT'], inplace=True)
data.sort_index(inplace=True)

data

HEAD   ORBIT_CNT  BX_COUNTER  TDC_MEAS
FPGA TDC_CHANNEL                                        
0    1               1  3869200678         227         0
     1               1  3869200803         297         0
     1               1  3869200745         834         0
     1               1  3869200290        1225         1
     1               1  3869200511        2509         1
...                ...         ...         ...       ...
1    139             1  3869200859        3555         0
     139             1  3869200374        3556         0
     139             1  3869201040        3556         0
     139             1  3869200312        3558         0
     139             1  3869200353        3559         0

[100000 rows x 4 columns]

2\. Find out the number of BX in a ORBIT (the value 'x').

In [61]:
X = data['BX_COUNTER'].max() + 1
print(f"The number of BX_COUNTERS in one TDC_MEAS is: {X}")

The number of BX_COUNTERS in one TDC_MEAS is: 3564


3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [111]:
time = data.loc[slice(None),'ORBIT_CNT':'TDC_MEAS']
time_max_TDC = time[time['TDC_MEAS'] == time['TDC_MEAS'].max()]
time_max_BX = time_max_TDC[time_max_TDC['BX_COUNTER'] == time_max_TDC['BX_COUNTER'].max()]
time_max_ORBIT = time_max_BX[time_max_BX['ORBIT_CNT'] == time_max_BX['ORBIT_CNT'].max()]

time_weight = numpy.array([25/30, 25, 25*X])

print(f"We estimate that these data have been collected for {numpy.dot(time_weight,numpy.array(time_max_ORBIT)[0].T) / 1e09}s")
print("We will check more precisely in the next answer, by reading the whole dataset")

We estimate that these data have been collected for 3.2270064708333335s
We will check more precisely in the next answer, by reading the whole dataset


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [112]:
data['TIME_NS'] = numpy.dot(time,time_weight)
print(f"We can now state that the data collecting took {data['TIME_NS'].max() / 1e09}s")

We can now state that the data collecting took 3.227007115s


5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [123]:
data['HEAD'] = numpy.random.binomial(1, 0.5, size=data.shape[0])
data

HEAD   ORBIT_CNT  BX_COUNTER  TDC_MEAS       TIME_NS
FPGA TDC_CHANNEL                                                      
0    1               1  3869200678         227         0  3.224340e+09
     1               1  3869200803         297         0  3.224341e+09
     1               0  3869200745         834         0  3.224355e+09
     1               0  3869200290        1225         1  3.224453e+09
     1               0  3869200511        2509         1  3.224486e+09
...                ...         ...         ...       ...           ...
1    139             1  3869200859        3555         0  3.224423e+09
     139             1  3869200374        3556         0  3.224423e+09
     139             0  3869201040        3556         0  3.224423e+09
     139             0  3869200312        3558         0  3.224423e+09
     139             1  3869200353        3559         0  3.224423e+09

[100000 rows x 5 columns]

6\. Create a new DataFrame that contains only the rows with HEAD=1.

In [124]:
new_data = data[data['HEAD'] == 1]
new_data

HEAD   ORBIT_CNT  BX_COUNTER  TDC_MEAS       TIME_NS
FPGA TDC_CHANNEL                                                      
0    1               1  3869200678         227         0  3.224340e+09
     1               1  3869200803         297         0  3.224341e+09
     1               1  3869200412         334         2  3.224520e+09
     1               1  3869200884        1173         2  3.224542e+09
     1               1  3869200265        1554         2  3.224551e+09
...                ...         ...         ...       ...           ...
1    139             1  3869200780        3549         0  3.224423e+09
     139             1  3869200606        3553         0  3.224423e+09
     139             1  3869200859        3555         0  3.224423e+09
     139             1  3869200374        3556         0  3.224423e+09
     139             1  3869200353        3559         0  3.224423e+09

[49920 rows x 5 columns]

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [125]:
!rm -r ./data